In [4]:
!pip install fsspec==2024.10.0

In [3]:
!pip install fsspec==2024.10.0

In [6]:
!pip install transformers datasets torch

In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import accuracy_score


In [8]:
# Load the IMDB dataset
dataset = load_dataset("imdb")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Function to compute accuracy
def compute_metrics(p):
    predictions, labels = p
    preds = torch.argmax(torch.tensor(predictions), axis=1)
    return {'accuracy': accuracy_score(labels, preds)}


In [12]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate every epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=tokenized_datasets['train'],  # training dataset
    eval_dataset=tokenized_datasets['test'],   # evaluation dataset
    compute_metrics=compute_metrics,     # evaluation metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [14]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to=None  # Disable logging to W&B
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
import os
os.environ["WANDB_MODE"] = "disabled"


In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Specify the output directory
    num_train_epochs=1,              # Reduce number of epochs to speed up training
    evaluation_strategy="epoch",     # Evaluate after each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directory for storing logs
    report_to=None,                  # Disable logging to W&B
    fp16=True                        # Use mixed precision training
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
training_args.num_train_epochs = 1  # Reduce number of epochs to speed up training


In [20]:
model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerN

In [22]:
training_args.per_device_train_batch_size = 8  # Use smaller batch size for faster training


In [24]:
training_args = TrainingArguments(
    output_dir='./results',          # Specify the output directory
    num_train_epochs=1,              # Reduce number of epochs to speed up training
    evaluation_strategy="epoch",     # Evaluate after each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directory for storing logs
    report_to=None,                  # Disable logging to W&B
    fp16=True                        # Use mixed precision training
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
from transformers import EarlyStoppingCallback

# Instead of re-initializing, update the existing training_args object
training_args.evaluation_strategy = "epoch"
training_args.save_strategy = "epoch"  # Save model after each epoch

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Early stopping
)


In [29]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset("imdb")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [31]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

# Define the metrics for evaluation
def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=1)
    return {"accuracy": accuracy_score(labels, preds)}

# Initialize the Trainer with the model, training arguments, and dataset
trainer = Trainer(
    model=model,  # The model to be trained
    args=training_args,  # The training arguments
    train_dataset=tokenized_datasets['train'],  # Training dataset
    eval_dataset=tokenized_datasets['test'],  # Evaluation dataset
    compute_metrics=compute_metrics  # The function to compute accuracy
)


In [32]:
# Start training the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.219100,0.226626,0.936440


TrainOutput(global_step=3125, training_loss=0.2785496044921875, metrics={'train_runtime': 983.9563, 'train_samples_per_second': 25.408, 'train_steps_per_second': 3.176, 'total_flos': 6577776384000000.0, 'train_loss': 0.2785496044921875, 'epoch': 1.0})

In [33]:
# Evaluate the model on the test dataset
results = trainer.evaluate()
print(results)


{'eval_loss': 0.22662560641765594, 'eval_accuracy': 0.93644, 'eval_runtime': 206.5077, 'eval_samples_per_second': 121.061, 'eval_steps_per_second': 1.893, 'epoch': 1.0}


In [39]:
import torch

# Check if GPU is available, if not, fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device (GPU or CPU)
model.to(device)

# Tokenize the text
text = "I will kiss you."
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move input tensors to the selected device (GPU or CPU)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Make predictions with the model
with torch.no_grad():
    logits = model(**inputs).logits

# Get the predicted sentiment (0 = negative, 1 = positive)
prediction = torch.argmax(logits, dim=-1).item()

# Print the prediction result
print("Predicted sentiment:", "Positive" if prediction == 1 else "Negative")



Predicted sentiment: Positive
